In [ ]:
"""
RAG (Retrieval Augmented Generation) with Claude
Install required packages:
pip install langchain langchain-anthropic langchain-community faiss-cpu sentence-transformers
"""

from langchain_anthropic import ChatAnthropic
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.schema import Document
import os

# Set your API key
os.environ["ANTHROPIC_API_KEY"] = "your-api-key-here"

# Initialize Claude
llm = ChatAnthropic(
    model="claude-sonnet-4-5-20250929",
    temperature=0.3,  # Lower temperature for factual retrieval
    max_tokens=1024
)

# ============================================================
# Example 1: Basic RAG with Simple Documents
# ============================================================
print("Example 1: Basic RAG Setup")
print("-" * 60)

# Sample documents
documents = [
    "Python is a high-level programming language known for its simplicity and readability.",
    "Machine learning is a subset of AI that enables systems to learn from data.",
    "LangChain is a framework for developing applications powered by language models.",
    "Claude is an AI assistant created by Anthropic that can help with various tasks.",
    "RAG combines retrieval systems with language models to provide accurate answers."
]

# Convert to Document objects
docs = [Document(page_content=doc) for doc in documents]

# Create embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Create vector store
vectorstore = FAISS.from_documents(docs, embeddings)

# Create retrieval chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

# Ask questions
questions = [
    "What is Python?",
    "Tell me about Claude",
    "What is RAG?"
]

for question in questions:
    result = qa_chain.invoke(question)
    print(f"Q: {question}")
    print(f"A: {result['result']}\n")

# ============================================================
# Example 2: RAG with Document Splitting
# ============================================================
print("Example 2: Processing Longer Documents")
print("-" * 60)

# Longer document
long_text = """
Python was created by Guido van Rossum and first released in 1991. 
It emphasizes code readability with its use of significant indentation.

Python supports multiple programming paradigms including structured, 
object-oriented, and functional programming. It has a comprehensive 
standard library, often described as having "batteries included."

Popular frameworks include Django for web development, NumPy for 
numerical computing, and TensorFlow for machine learning. Python is 
widely used in data science, artificial intelligence, web development,
automation, and scientific computing.
"""

# Split the text into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50,
    length_function=len
)

chunks = text_splitter.create_documents([long_text])
print(f"Split document into {len(chunks)} chunks")

# Create vector store
vectorstore = FAISS.from_documents(chunks, embeddings)

# Query
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever()
)

result = qa_chain.invoke("What frameworks does Python have?")
print(f"\nQ: What frameworks does Python have?")
print(f"A: {result['result']}\n")

# ============================================================
# Example 3: RAG with Custom Prompts
# ============================================================
print("Example 3: Custom RAG Prompt")
print("-" * 60)

# Create custom prompt template
prompt_template = """Use the following pieces of context to answer the question. 
If you don't know the answer, just say you don't know. Don't make up an answer.
Keep the answer concise and cite which part of the context you used.

Context: {context}

Question: {question}

Answer:"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# Create chain with custom prompt
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": PROMPT}
)

result = qa_chain.invoke("Who created Python?")
print(f"Q: Who created Python?")
print(f"A: {result['result']}\n")

# ============================================================
# Example 4: RAG with Multiple Document Sources
# ============================================================
print("Example 4: Multiple Document Sources")
print("-" * 60)

# Different document sources
company_docs = [
    Document(
        page_content="Our company was founded in 2020 and specializes in AI solutions.",
        metadata={"source": "about.txt", "type": "company_info"}
    ),
    Document(
        page_content="We offer 24/7 customer support via email and phone.",
        metadata={"source": "support.txt", "type": "support_info"}
    ),
    Document(
        page_content="Our main product is an AI-powered analytics platform.",
        metadata={"source": "products.txt", "type": "product_info"}
    )
]

# Create vector store with metadata
vectorstore = FAISS.from_documents(company_docs, embeddings)

# Retrieve with source information
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

result = qa_chain.invoke("What does the company do?")
print(f"Q: What does the company do?")
print(f"A: {result['result']}")
print(f"\nSources used:")
for doc in result['source_documents']:
    print(f"  - {doc.metadata['source']}: {doc.metadata['type']}\n")

# ============================================================
# Example 5: RAG with Similarity Search
# ============================================================
print("Example 5: Direct Similarity Search")
print("-" * 60)

# Create a knowledge base
knowledge_base = [
    "The capital of France is Paris.",
    "The capital of Germany is Berlin.",
    "The capital of Italy is Rome.",
    "The capital of Spain is Madrid.",
    "The capital of Portugal is Lisbon."
]

docs = [Document(page_content=text) for text in knowledge_base]
vectorstore = FAISS.from_documents(docs, embeddings)

# Perform similarity search
query = "What is the capital of France?"
relevant_docs = vectorstore.similarity_search(query, k=2)

print(f"Query: {query}")
print("Most relevant documents:")
for i, doc in enumerate(relevant_docs, 1):
    print(f"{i}. {doc.page_content}")

# Use with Claude
context = "\n".join([doc.page_content for doc in relevant_docs])
response = llm.invoke(f"Based on this context: {context}\n\nQuestion: {query}")
print(f"\nClaude's answer: {response.content}\n")

# ============================================================
# Example 6: RAG with File Loading
# ============================================================
print("Example 6: Loading from Text Files")
print("-" * 60)

# Simulate loading from files (in practice, use actual files)
file_contents = {
    "python_basics.txt": """
    Python is an interpreted, high-level programming language.
    It uses dynamic typing and garbage collection.
    Python's design philosophy emphasizes code readability.
    """,
    "python_libraries.txt": """
    Popular Python libraries include:
    - NumPy for numerical computing
    - Pandas for data manipulation
    - Matplotlib for visualization
    - Scikit-learn for machine learning
    """
}

# Create documents with metadata
file_docs = []
for filename, content in file_contents.items():
    file_docs.append(
        Document(
            page_content=content,
            metadata={"source": filename}
        )
    )

# Split and create vector store
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=20)
split_docs = text_splitter.split_documents(file_docs)

vectorstore = FAISS.from_documents(split_docs, embeddings)

# Query with source tracking
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

result = qa_chain.invoke("What Python libraries are mentioned?")
print(f"Q: What Python libraries are mentioned?")
print(f"A: {result['result']}")
print(f"Source: {result['source_documents'][0].metadata['source']}\n")

# ============================================================
# Example 7: Advanced RAG with MMR (Maximum Marginal Relevance)
# ============================================================
print("Example 7: Using MMR for Diverse Results")
print("-" * 60)

tech_docs = [
    "Python is great for web development with frameworks like Django and Flask.",
    "Python excels in data science with libraries like Pandas and NumPy.",
    "Python is used in machine learning with TensorFlow and PyTorch.",
    "JavaScript is the language of the web and runs in browsers.",
    "Java is widely used for enterprise applications and Android development.",
]

docs = [Document(page_content=text) for text in tech_docs]
vectorstore = FAISS.from_documents(docs, embeddings)

# Use MMR to get diverse results
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3, "fetch_k": 5}
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever
)

result = qa_chain.invoke("What is Python used for?")
print(f"Q: What is Python used for?")
print(f"A: {result['result']}\n")

# ============================================================
# Example 8: RAG with Conversation Memory
# ============================================================
print("Example 8: Conversational RAG")
print("-" * 60)

from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# Create memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"
)

# Create conversational chain
conv_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    memory=memory,
    return_source_documents=True
)

# Have a conversation
questions = [
    "What programming languages are mentioned?",
    "Which one is best for web development?",
    "What about mobile development?"
]

for question in questions:
    result = conv_chain.invoke({"question": question})
    print(f"Q: {question}")
    print(f"A: {result['answer']}\n")

# ============================================================
# Example 9: Evaluating RAG Results
# ============================================================
print("Example 9: Checking Retrieved Context")
print("-" * 60)

query = "What is machine learning?"
retrieved_docs = vectorstore.similarity_search(query, k=3)

print(f"Query: {query}")
print(f"Retrieved {len(retrieved_docs)} documents:")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"\n{i}. {doc.page_content}")

# Show relevance scores
docs_with_scores = vectorstore.similarity_search_with_score(query, k=3)
print("\nWith relevance scores:")
for doc, score in docs_with_scores:
    print(f"Score: {score:.4f} - {doc.page_content[:50]}...")

# ============================================================
# Example 10: Complete RAG Application
# ============================================================
print("\n" + "="*60)
print("Example 10: Complete RAG Application")
print("="*60)

class DocumentQA:
    """Complete RAG application class"""
    
    def __init__(self, api_key):
        os.environ["ANTHROPIC_API_KEY"] = api_key
        self.llm = ChatAnthropic(
            model="claude-sonnet-4-5-20250929",
            temperature=0.3
        )
        self.embeddings = HuggingFaceEmbeddings()
        self.vectorstore = None
    
    def add_documents(self, texts, metadata_list=None):
        """Add documents to the knowledge base"""
        if metadata_list:
            docs = [
                Document(page_content=text, metadata=meta)
                for text, meta in zip(texts, metadata_list)
            ]
        else:
            docs = [Document(page_content=text) for text in texts]
        
        if self.vectorstore is None:
            self.vectorstore = FAISS.from_documents(docs, self.embeddings)
        else:
            self.vectorstore.add_documents(docs)
        
        print(f"Added {len(docs)} documents to knowledge base")
    
    def query(self, question, return_sources=False):
        """Query the knowledge base"""
        if self.vectorstore is None:
            return "No documents in knowledge base"
        
        qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            retriever=self.vectorstore.as_retriever(search_kwargs={"k": 3}),
            return_source_documents=return_sources
        )
        
        result = qa_chain.invoke(question)
        
        if return_sources:
            return {
                "answer": result['result'],
                "sources": [doc.page_content for doc in result['source_documents']]
            }
        return result['result']
    
    def save_index(self, path):
        """Save the vector store to disk"""
        if self.vectorstore:
            self.vectorstore.save_local(path)
            print(f"Index saved to {path}")
    
    def load_index(self, path):
        """Load a vector store from disk"""
        self.vectorstore = FAISS.load_local(
            path,
            self.embeddings,
            allow_dangerous_deserialization=True
        )
        print(f"Index loaded from {path}")

# Use the application
qa_app = DocumentQA("your-api-key-here")

# Add documents
qa_app.add_documents([
    "Artificial Intelligence is the simulation of human intelligence by machines.",
    "Machine Learning is a subset of AI that learns from data.",
    "Deep Learning uses neural networks with multiple layers.",
    "Natural Language Processing enables computers to understand human language."
])

# Query
answer = qa_app.query("What is the difference between AI and ML?")
print(f"\nQuestion: What is the difference between AI and ML?")
print(f"Answer: {answer}")

# Query with sources
result = qa_app.query("What is deep learning?", return_sources=True)
print(f"\nQuestion: What is deep learning?")
print(f"Answer: {result['answer']}")
print(f"Sources: {len(result['sources'])} documents retrieved")

print("\n" + "="*60)
print("RAG setup complete! Try running these examples.")
print("="*60)